<a href="https://colab.research.google.com/github/nahbos/AUT-Language-Understanding/blob/main/Proj/farsi_poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sobhan Moradian Daghigh
Project: NLG - Poem

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.2 MB/s eta 0:00:00


In [2]:
import json
import os
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm

In [3]:
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Proj/Datasets/train.csv
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Proj/Datasets/test.csv

--2023-02-23 14:50:18--  https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Proj/Datasets/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 705449 (689K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 688.92K  --.-KB/s    in 0.05s   

2023-02-23 14:50:18 (14.7 MB/s) - ‘train.csv’ saved [705449/705449]

--2023-02-23 14:50:19--  https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Proj/Datasets/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [5]:
train.head()

,cat_title,v_order,v_position,poem_text,poet_name,poem_id
0,رباعی,1,0,جز نقش تو در نظر نیامد ما را,حافظ,1
1,رباعی,2,1,جز کوی تو رهگذر نیامد ما را,حافظ,1
2,رباعی,3,0,خواب ارچه خوش آمد همه را در عهدت,حافظ,1
3,رباعی,4,1,حقا که به چشم در نیامد ما را,حافظ,1
4,رباعی,1,0,بر گیر شراب طرب‌انگیز و بیا,حافظ,2


In [6]:
poets = list(train['cat_title'].unique())
poets

['رباعی', 'قطعه', 'قصیده', 'غزل', '<unk>']

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8610 entries, 0 to 8609
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cat_title   8610 non-null   object
 1   v_order     8610 non-null   int64 
 2   v_position  8610 non-null   int64 
 3   poem_text   8610 non-null   object
 4   poet_name   8610 non-null   object
 5   poem_id     8610 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 403.7+ KB


In [8]:
list(train['cat_title']).count('<unk>')

496

In [9]:
import pandas as pd
import numpy as np

from pathlib import Path

import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config

from IPython import display

In [10]:
model_name_or_path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

tokenizer.add_special_tokens({
    "bos_token": '<bos>',
    "eos_token": '<eos>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

2

In [11]:
tokenizer.encode('<bos>')[0]

42001

In [12]:
print(tokenizer.encode("به به حسین اقا حال شما چطوره؟"))
print(tokenizer.encode(""))
print(tokenizer.encode(""))
print(tokenizer.encode(""))
print(tokenizer.encode("<|startoftext|>"))
print(tokenizer.encode(""))

[490, 303, 3357, 35343, 532, 512, 34505, 677]
[]
[]
[]
[6]
[]


In [13]:
texts = train['poem_text'].values.tolist()
max_seq = max([len(tokenizer.encode(text)) for text in texts])

print(f'The longest text is {max_seq} tokens long.')
print('This is for one hemistich!')

The longest text is 15 tokens long.
This is for one hemistich!


In [48]:
max_seq = 15

In [15]:
from torch.utils.data import Dataset
import torch
torch.manual_seed(42)


class MTGDataset(Dataset):
    def __init__(self, hemistichs1, hemistichs2, tokenizer, max_length=1024):

        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        for hemistich1, hemistich2 in zip(hemistichs1, hemistichs2):
            encodings_dict = tokenizer('' + hemistich1 + '<sep>' + hemistich2 + '',
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")

            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [16]:
groups = train.groupby(by='v_position')
hemistichs1, hemistichs2 = [], []
for grp, pdf in groups:
    if grp == 0:
        hemistichs1 = list(pdf['poem_text'])
    else:
        hemistichs2 = list(pdf['poem_text'])

In [17]:
hemistichs1[0]

'جز نقش تو در نظر نیامد ما را'

In [18]:
hemistichs2[0]

'جز کوی تو رهگذر نیامد ما را'

In [19]:
from torch.utils.data import random_split

dataset = MTGDataset(hemistichs1, hemistichs2, tokenizer, max_length=max_seq)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {len(train_dataset)} samples for training, and {len(val_dataset)} samples for validation testing'

'There are 3874 samples for training, and 431 samples for validation testing'

In [20]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=8
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=8
)

In [21]:
config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer.encode('<bos>')[0], 
    eos_token_id=tokenizer.encode('<eos>')[0], 
    pad_token_id=tokenizer.encode('<pad>')[0],
    unk_token_id=tokenizer.encode('<unk>')[0],
)

In [22]:
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin" -P /content/gpt2/
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/config.json" -P /content/gpt2/

--2023-02-23 14:50:32--  https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/HooshvareLab/gpt2-fa/46b0b806c740a0f0a9f056f5574c5fa896166fe844945fd3c849bf34365e5060?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&Expires=1677423032&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL0hvb3NodmFyZUxhYi9ncHQyLWZhLzQ2YjBiODA2Yzc0MGEwZjBhOWYwNTZmNTU3NGM1ZmE4OTYxNjZmZTg0NDk0NWZkM2M4NDliZjM0MzY1ZTUwNjA%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNjc3NDIzMDMyfX19XX0_&Signature=H3GNEXrULK9kVaoXZTTS0iLjh9vRdzqd6cfdcT4X9rRIf7

In [23]:
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/config.json" -P /content/gpt2/

--2023-02-23 14:50:36--  https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/config.json
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 808 [text/plain]
Saving to: ‘/content/gpt2/config.json.1’

config.json.1       100%[===================>]     808  --.-KB/s    in 0s      

2023-02-23 14:50:36 (209 MB/s) - ‘/content/gpt2/config.json.1’ saved [808/808]



In [24]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('./gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("./gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [75]:
epochs = 14
warmup_steps = 1e2
sample_every = 100

In [26]:
from transformers import AdamW

# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters.
optimizer = AdamW(
    model.parameters(),
    lr=5e-4,
    eps=1e-8
)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)

In [36]:
h1 = hemistichs1[np.random.randint(0, len(hemistichs1))]
sample_input = f"{h1}<|startoftext|>"
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.generate(
    input_ids=sample_input_ids,
    # bos_token_id=random.randint(1, len(tokenizer.get_vocab())),
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=1
)
# for i, sample_output in enumerate(sample_outputs):
gen_sample_output = tokenizer.decode(sample_outputs[0], skip_special_tokens=False)
    # gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    # gen_sample_output = gen_sample_output.replace("", "")
    # gen_sample_output = gen_sample_output.replace("", "")
    # gen_sample_output = gen_sample_output.replace("", "\n")

print(f'Example output: {gen_sample_output}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


ای نسیم سحر آرامگه یار کجاست<|startoftext|>
Example output: ای نسیم سحر آرامگه یار کجاست<|startoftext|> منان، از بس که عاشق آن دو هستم، به دور از هر گونه قضاوت و قضاوت، هر جا که هستی، جایی، کسی مرا نمی‌شناسد. چون مرا به یاد دارند که به یاد داشته


In [76]:
import random
import time
import datetime
from tqdm import tqdm


def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))


total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in tqdm(range(0, epochs), position=0):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 100 batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print()
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')

            model.eval()

            sample_h1 = hemistichs1[np.random.randint(0, len(hemistichs1))]
            sample_input = f"{sample_h1}<|startoftext|>"
            sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
            sample_input_ids = sample_input_ids.to(device)

            sample_outputs = model.generate(
                input_ids=sample_input_ids,
                do_sample=True,
                top_k=50,
                max_length=max_seq,
                top_p=0.95,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id
            )
            
            for i, output in enumerate(sample_outputs):
                o = tokenizer.decode(output, skip_special_tokens=False)
                o = o.replace("<|startoftext|>", "    ")

            print()
            print(f'Example output: {o}')

            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print()
    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')
    print()

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print()
    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')
    print()

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

  0%|          | 0/14 [00:00<?, ?it/s]

Beginning epoch 1 of 14


 21%|██        | 101/485 [00:16<01:07,  5.69it/s]


Batch 100 of 485. Loss:0.5142054557800293. Time:0:00:16

Example output: ببرد از من قرار و طاقت و هوش    کان تحمل رنج بیهوده را


 41%|████▏     | 201/485 [00:32<00:50,  5.65it/s]

...


Batch 400 of 485. Loss:0.5312782526016235. Time:0:01:05

Example output: بشد که یاد خوشش باد روزگار وصال    به کام دل دانا بود و


100%|██████████| 485/485 [01:18<00:00,  6.15it/s]



Average Training Loss: 0.5070128160039175. Epoch time: 0:01:19



100%|██████████| 14/14 [18:54<00:00, 81.06s/it]


Validation loss: 1.7046456403202481. Validation Time: 0:00:02

Total training took 0:18:55


In [142]:
def generator(sample_h1, max_length=128, num_return_sequences=3, verbose=0):
    model.eval()
    prompt = f"{sample_h1}<|startoftext|>"

    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    decoded_outputs = model.generate(
        generated,
        do_sample=True,
        top_k=50,
        max_length=20, 
        top_p=0.95,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )

    for i, output in enumerate(decoded_outputs):
        o = tokenizer.decode(output, skip_special_tokens=False)
        out = o.replace("<|startoftext|>", "    ")
        h2 = o.split("<|startoftext|>")[-1]
        if verbose == 0:
            print(out)

    return h2

In [85]:
sample_h1 = hemistichs1[1]
generator(sample_h1, max_length=max_seq, num_return_sequences=1)

خواب ارچه خوش آمد همه را در عهدت    به عزم سوی باغ آمد و آمد باغبانی


In [86]:
sample_h1 = hemistichs1[1]
generator(sample_h1, max_length=max_seq, num_return_sequences=1)

خواب ارچه خوش آمد همه را در عهدت    ز لعل لبت در دهان بگردان و


In [87]:
sample_h1 = hemistichs1[1]
generator(sample_h1, max_length=max_seq, num_return_sequences=1)

خواب ارچه خوش آمد همه را در عهدت    بجز خیال آن یار که ننوشد


In [89]:
for _ in range(10):
    generator(hemistichs1[1], max_length=max_seq, num_return_sequences=1)

خواب ارچه خوش آمد همه را در عهدت    حیف باشد که بی سر و سامان شوی
خواب ارچه خوش آمد همه را در عهدت    به دور دهانت از خواب برنتابم
خواب ارچه خوش آمد همه را در عهدت    زین تغابن که گویند همه خواب خوش است
خواب ارچه خوش آمد همه را در عهدت    وز آن به که بیاسای تو هر
خواب ارچه خوش آمد همه را در عهدت    به یک جرعه با سحر از سحر بگذرم
خواب ارچه خوش آمد همه را در عهدت    در میان خواب دوای سحر آه به درن
خواب ارچه خوش آمد همه را در عهدت    تزاع کاین دل ارچه خوش شد
خواب ارچه خوش آمد همه را در عهدت    از بختت که در خواب خوش آمد<pad>
خواب ارچه خوش آمد همه را در عهدت    که من در خواب نهخواب نه به سر
خواب ارچه خوش آمد همه را در عهدت    گر چه خوش آمد همه را در عهدت


In [93]:
for _ in range(10):
    generator(hemistichs1[3], max_length=max_seq, num_return_sequences=1)

مشنو سخن خصم که بنشین و مرو    هر که خوش لهجه ادا می‌گوید زبان
مشنو سخن خصم که بنشین و مرو    بشنو ز من ایشان که با ما
مشنو سخن خصم که بنشین و مرو    ز زبان من برون آمد و در میان آمد
مشنو سخن خصم که بنشین و مرو    گفتمت مگر گو بیا که سوسن آزاده کنی
مشنو سخن خصم که بنشین و مرو    تو گویی که در پرده شطرنج جمشید و بهمن
مشنو سخن خصم که بنشین و مرو    ز لب من بوسه بر لب یار بخش<pad>
مشنو سخن خصم که بنشین و مرو    به یک جرعه نوش و یک پیمانه ننو
مشنو سخن خصم که بنشین و مرو    ننهان خور و نه اندیشه آنان که
مشنو سخن خصم که بنشین و مرو    بشنو سخن خصم چون بنشین و
مشنو سخن خصم که بنشین و مرو    بازآ که دگر سخن خصم که گو


In [95]:
for _ in range(10):
    generator(hemistichs1[4], max_length=max_seq, num_return_sequences=1)

ماهی که قدش به سرو می‌ماند راست    بر لب رود از خط هلالی<pad>چو جم
ماهی که قدش به سرو می‌ماند راست    چو قدش به سرو خرامان می‌
ماهی که قدش به سرو می‌ماند راست    به صد گونه هواداری می‌کند از راست
ماهی که قدش به سرو می‌ماند راست    زین بحر می‌گذرد از جویبار حسن<pad>
ماهی که قدش به سرو می‌ماند راست    می‌رود چون قدح پربده
ماهی که قدش به سرو می‌ماند راست    گرد چمن ناز از خط مشکین نقاب انداز<pad>
ماهی که قدش به سرو می‌ماند راست    بلبلان و به آواز زهره می‌
ماهی که قدش به سرو می‌ماند راست    ز خوش به دمی که درآید بازآی باز
ماهی که قدش به سرو می‌ماند راست    ز قدش به سرو می‌ماند راست
ماهی که قدش به سرو می‌ماند راست    زین چمن عارضش به سرو دگر<pad>


### Evaluate

In [96]:
groups = test.groupby(by='v_position')
hemistichs1_test, hemistichs2_test = [], []
for grp, pdf in groups:
    if grp == 0:
        hemistichs1_test = list(pdf['poem_text'])
    else:
        hemistichs2_test = list(pdf['poem_text'])

In [97]:
for _ in range(10):
    generator(hemistichs1_test[0], max_length=max_seq, num_return_sequences=1)

گفتم که لبت، گفت لبم آب حیات    گفتا که این قطره‌های شبنم باشد
گفتم که لبت، گفت لبم آب حیات    گفت این گفت و شنفتاده‌ام
گفتم که لبت، گفت لبم آب حیات    گفتا که در این آب به سر نمی
گفتم که لبت، گفت لبم آب حیات    گفتا محض حقیقت تو گویی بکند<pad><pad>
گفتم که لبت، گفت لبم آب حیات    گفت حافظ ز آب حیات به من بازرسان<pad>
گفتم که لبت، گفت لبم آب حیات    گفتا خوشا دمی که با تو آب
گفتم که لبت، گفت لبم آب حیات    گفتا خنک کن و از آب لطف زاده
گفتم که لبت، گفت لبم آب حیات    گفت حافظ دیوانه را همین آب حیات است<pad>
گفتم که لبت، گفت لبم آب حیات    گفتا که با وی گفت‌وگو آشنا
گفتم که لبت، گفت لبم آب حیات    گفتا مگوی غم و حسرت من<pad>


In [99]:
for _ in range(10):
    generator(hemistichs1_test[100], max_length=max_seq, num_return_sequences=1)

رندان تشنه لب را آبی نمی‌دهد کس    به دست کسی که این تشنه نیست این آب
رندان تشنه لب را آبی نمی‌دهد کس    ز حقه سر آن دو دیده هیچش به
رندان تشنه لب را آبی نمی‌دهد کس    لب تشنه را آب لب جز آب نیست ره
رندان تشنه لب را آبی نمی‌دهد کس    بگو بسوز که من در دام این ن
رندان تشنه لب را آبی نمی‌دهد کس    ای خضر پی خجسته مدد کن آن قطره خون
رندان تشنه لب را آبی نمی‌دهد کس    عاقبت این است خدایا باش که او
رندان تشنه لب را آبی نمی‌دهد کس    فغان که ما در این کار بی‌
رندان تشنه لب را آبی نمی‌دهد کس    وز لب چشمه زنخدان تشنه می‌
رندان تشنه لب را آبی نمی‌دهد کس    ور نه‌ای دل که به جگر تشنه
رندان تشنه لب را آبی نمی‌دهد کس    چو لاله کاسه سر چشمه را جوی نمی‌


In [117]:
for _ in range(10):
    generator(hemistichs1_test[65], max_length=max_seq, num_return_sequences=1)

صراحی ای و حریفی گرت به چنگ افتد    که روز قیامت ز می صاف کن راز چو
صراحی ای و حریفی گرت به چنگ افتد    اگر به عشوه چنگ آورد سخنی با من
صراحی ای و حریفی گرت به چنگ افتد    که به رقص آورد و جامی به خون جگر
صراحی ای و حریفی گرت به چنگ افتد    ور از میان من و تو در میانه من
صراحی ای و حریفی گرت به چنگ افتد    که من غلام غلام غلام و از سخند
صراحی ای و حریفی گرت به چنگ افتد    کز غمش به تنگ آمده به لب جام
صراحی ای و حریفی گرت به چنگ افتد    تو سخن سخت و حریف اندر دهان به من
صراحی ای و حریفی گرت به چنگ افتد    که در این کار جز به سیم و زر
صراحی ای و حریفی گرت به چنگ افتد    چه خوش آمد که این ترانه سر آمد و
صراحی ای و حریفی گرت به چنگ افتد    که تا به صبحدم نمی‌آید بوی


In [169]:
from nltk.translate.bleu_score import sentence_bleu

def cal_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
    return score

In [145]:
scores = []
for h1_test, h2_test in zip(hemistichs1_test, hemistichs2_test):
    output = generator(h1_test, max_length=max_seq, num_return_sequences=1, verbose=1)
    score = cal_bleu(reference=h2_test, candidate=output)
    scores.append(score)
  
np.average(scores)

0.05107688098228099

In [173]:
scores = []
outputs = []
for h1_test, h2_test in zip(hemistichs1_test, hemistichs2_test):
    output = generator(h1_test, max_length=max_seq, num_return_sequences=1, verbose=1)
    outputs.append(output)
    score = cal_bleu(reference=h2_test, candidate=output)
    scores.append(score)
  
round(np.average(scores), 3)

0.056

In [176]:
ind = np.argmax(scores)
hemistichs1_test[ind].replace('\u200c', ' '), hemistichs2_test[ind], outputs[ind].replace('<pad>', '')

('بکن معامله ای وین دل شکسته بخر',
 'که با شکستگی ارزد به صد هزار درست',
 'که جام می به صد شعبده باز دهم')

Finitoo